# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Импортируем библиотеки

In [ ]:
import pandas as pd
import numpy as np
import time
from IPython.display import display
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Откроем датафрейм и посмотрим на первые значения в нем

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')
display(df.head())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Посмотрим типы данных и количество записей

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


### Вывод:
 - В таблице 5 столбцов, 3214 строк
 - Пропуски отсутствуют
 - В столбцах **calls, minutes, messages, mb_used** тип даных `float64`, в столбце **is_ultra** тип данных `int64`
 - Целевой признак `is_ultra` категориальный, бинарная классификация.

## Разбейте данные на выборки

В переменной `features` сохраним признаки, в переменной `target` будет целевой признак

In [ ]:
features = df.drop(['is_ultra', 'calls'],axis=1)
target = df['is_ultra']

Создадим тестовую выборку

In [ ]:
features_80, features_test, target_80, target_test = train_test_split(
    features, target, test_size=0.2, random_state=2034)

Создадим валидационную выборку и обучающие данные

In [ ]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_80, target_80, test_size=0.25, random_state=2034)

### Вывод:
 - Получили 3 набора данных:
   - обучающие 0.6
   - валидационные 0.2
   - тестовые 0.2.

## Исследуйте модели

Модель дерево решений DecisionTreeClassifier
Напишем функцию, которая будет менять глубину дерева 1-10 и выбирать лучшую модель дерева решений на основе accuracy

In [ ]:
model_tree = None
best_result_1 = 0

for depth in range(1, 11):
    model_1 = DecisionTreeClassifier(random_state=2034, max_depth=depth)
    model_1.fit(features_train,target_train)
    result_1 = model_1.score(features_valid,target_valid)
    if result_1 > best_result_1:
        model_tree = model_1
        best_result_1 = result_1

display("Accuracy наилучшей модели на валидационной выборке:", best_result_1)

'Accuracy наилучшей модели на валидационной выборке:'

0.8055987558320373

По заданию было нужно довести долю правильных ответов по крайней мере до 0.75. Задание выполнено.

Модель логистическая регрессия LogisticRegression

In [ ]:
model_log = LogisticRegression(random_state=2034, solver='lbfgs')
model_log.fit(features_train, target_train)
result_2 = model_log.score(features_valid, target_valid)

display("Accuracy  модели логистической регрессии на валидационной выборке:", result_2)

'Accuracy  модели логистической регрессии на валидационной выборке:'

0.7060653188180405

Полученная точность 0.70 ниже необходимой по заданию 0.75

Модель случаный лес RandomForestClassifier

In [ ]:
model_forest = None
best_result = 0
for est in range(1, 51):
    model = RandomForestClassifier(random_state=2034, n_estimators=est)
    model.fit(features_train,target_train)
    result = model.score(features_valid,target_valid)
    if result > best_result:
        model_forest = model
        best_result = result

display("Accuracy наилучшей модели на валидационной выборке:", best_result)

'Accuracy наилучшей модели на валидационной выборке:'

0.8102643856920684

По заданию было нужно довести долю правильных ответов по крайней мере до 0.75. Задание выполнено.

У моделей дерево решений и случайный лес схожая точность, определим скорость выполнения (время работы)

In [ ]:
start_time = time.time()
model_forest.predict(features_valid)
display("Время выполнения решения model_forest %s секунд" % (time.time() - start_time))
start_time = time.time()
model_tree.predict(features_valid)
display("Время выполнения решения model_tree %s секунд" % (time.time() - start_time))

'Время выполнения решения model_forest 0.01715683937072754 секунд'

'Время выполнения решения model_tree 0.0026705265045166016 секунд'

### Вывод:
 - Выбирая из двух моделей, выбрали ту, которая выполняется быстрее, остановились на `model_tree`

## Проверьте модель на тестовой выборке

Получим предсказания по тестовой выборке:

In [ ]:
predictions_test = model_tree.predict(features_test)

Получим предсказания по обучающей выборке:

In [ ]:
predictions_train = model_tree.predict(features_train)

Получим точность модели на обучающей выборке:

In [ ]:
accuarry_train = accuracy_score(target_train, predictions_train)

Получим точность модели на тестовой выборке:

In [ ]:
accuarry_test = accuracy_score(target_test, predictions_test)

display('Accuracy')
display('Обучающая выборка:', accuarry_train)
display('Тестовая выборка:', accuarry_test)

'Accuracy'

'Обучающая выборка:'

0.7956431535269709

'Тестовая выборка:'

0.7947122861586314

Вывод:
 - На обучающей и тестовой выборке получили долю правильных ответов 0.79

## (бонус) Проверьте модели на адекватность

Определим какой самый часто встречающейся класс в тестовой выборке:

In [ ]:
target_test.value_counts()

0    444
1    199
Name: is_ultra, dtype: int64

Передадим моделе данные состоящие только из 0 и сделаем проверку

In [ ]:
predictions_adequacy = [0]*target_test.count()
adequacy_test = accuracy_score(target_test, predictions_adequacy)

display('Accuracy')
display('Accuracy при одинаковых ответах:', adequacy_test)
display('Тестовая выборка:', accuarry_test)

'Accuracy'

'Accuracy при одинаковых ответах:'

0.6905132192846034

'Тестовая выборка:'

0.7947122861586314

### Вывод:
 - Модель обучилась

## Общий вывод:  
Перед нами стояла задача для оператора мобильной связи «Мегалайн»: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Мы загрузили и изучили данные.
Данные были разбиты на три выборки, произведено исследование моделей.
Построили модель для задачи классификации, которая выберет подходящий тариф. Построили модель с максимально большим значением accuracy 0.81.  
Рекомендация для компании использовать модель случаный лес RandomForestClassifier.